# 第4章 朴素贝叶斯

借鉴Sklearn中一共有3个朴素贝叶斯的分类算法类：
- GaussianNB 高斯模型
- MultinomialNB 多项式模型
- BernoulliNB 伯努利模型

本算法实现GaussianNB朴素贝叶斯

朴素贝叶斯法是基于贝叶斯定理与特征条件独立假设的分类方法。其是一种典型的生成式学习方法。

生成式模型评估对象的是最大化联合概率p(x,y)并对其建模；

判别式模型评估对象是最大化条件概率p(y|x)并直接对其建模。

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [2]:
# load data
def iris_data():
    iris = load_iris()
    iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
    iris_df['label'] = iris.target
    iris_df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(iris_df.iloc[:100, :])
    return data[:, :-1], data[:, -1]

In [3]:
# split train set and test set
X, y = iris_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## GaussianNB
特征的可能性被假设为高斯概率密度函数：$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$

数学期望为：$\mu$

方差为：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

参考：https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

- Python 字典(Dictionary) items() 函数以列表返回可遍历的(键, 值) 元组数组。

In [4]:
class NaiveBayes:
    def __init__(self):
        self.model = None
        
    # Mathematical Expectation
    @staticmethod # decorative static method
    def mean(X):
        return sum(X) / float(len(X))
    
    # Standard deviation
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x-avg, 2) for x in X]) / float(len(X)))
    
    # Probability density function
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
    
    # Processing X_train
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(* train_data)]
        return summaries
    
    # Find mathematical expectations and standard deviations by category
    def fit(self, X, y):
        labels = list(set(y))
        data = {label:[] for label in labels}
        for f, label in zip(X, y):
            data[label].append(f)
        self.model = {label: self.summarize(value) for label, value in data.items()}
        return 'GaussianNB train done!'
    
    # Calculation the probability
    def calculate_probabilities(self, input_data):
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(input_data[i], mean, stdev)
        return probabilities
    
    # pradect the category
    def predict(self, X_test):
        label = sorted(self.calculate_probabilities(X_test).items(), key=lambda x: x[-1])[-1][0]
        return label
    
    # predict score
    def score(self, X_test, y_test):
        right_count = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right_count += 1
        return right_count / float(len(X_test))

In [5]:
model = NaiveBayes()

In [6]:
model.fit(X_train, y_train)

'GaussianNB train done!'

In [7]:
print( model.predict([4.4, 3.2, 1.3, 0.2]) )

0.0


In [8]:
model.score(X_test, y_test)

1.0

## sklearn.naive_bayes

In [9]:
from sklearn.naive_bayes import GaussianNB

In [10]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [11]:
clf.score(X_test, y_test)

1.0

In [12]:
clf.predict([[4.4, 3.2, 1.3, 0.2]])

array([0.])